Sometimes when we get data set through the Microsoft  vision or  download  on webset. But it is hard to seperat to dummy column. Look at the data sample.cvs and you will know some common forms  in the orgional clolums .
This tutorial will teach you how to use the function to process these columns to dummy columns.

In [ ]:
from google.colab import files
files.upload()
#please upload the Data_Clean_Healper.py and data sample.csv

Saving amnestyus_face_raw.csv to amnestyus_face_raw (4).csv


{'amnestyus_face_raw.csv': b'Unnamed: 0,picture_name,smile,gender,age,facialHair,glasses,emotion,blur,exposure,noise,makeup,accessories,occlusion,hair\n0,13,[],[],[],[],[],[],[],[],[],[],[],[],[]\n1,14,0.997,female,40.0,"{\'moustache\': 0.0, \'beard\': 0.0, \'sideburns\': 0.0}",NoGlasses,"{\'anger\': 0.0, \'contempt\': 0.0, \'disgust\': 0.0, \'fear\': 0.0, \'happiness\': 0.997, \'neutral\': 0.003, \'sadness\': 0.0, \'surprise\': 0.0}","{\'blurLevel\': \'low\', \'value\': 0.21}","{\'exposureLevel\': \'goodExposure\', \'value\': 0.64}","{\'noiseLevel\': \'low\', \'value\': 0.25}","{\'eyeMakeup\': True, \'lipMakeup\': True}",[],"{\'foreheadOccluded\': False, \'eyeOccluded\': False, \'mouthOccluded\': False}","{\'bald\': 0.05, \'invisible\': False, \'hairColor\': [{\'color\': \'brown\', \'confidence\': 0.98}, {\'color\': \'black\', \'confidence\': 0.98}, {\'color\': \'other\', \'confidence\': 0.17}, {\'color\': \'gray\', \'confidence\': 0.17}, {\'color\': \'red\', \'confidence\': 0.11}, {\

In [ ]:
import pandas as pd
import numpy as np
import pandas
import re
from pandas import Series
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import re
import spacy
#process the column like ['QLD', 'climateaction', 'windenergy', 'wind', 'renewableenergy', 'olarenergy', 'cleanenergy', 'solarpower']
#make it to dummy variable and drop the dummy column if the sum in the dummy column less than 4
def seperate_tag_text(column_name,df,drop_number):
  #print(df.columns)
  column=df[column_name]
  my_list=[]
  for i in column:
    tmp=[]
    for j in range(len(eval(i))):
      tmp.append(i[j])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  #columns.fit_transform(column)
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      del df_2[column]
  df_2_2 = df.join(df_2, how='outer')
  #print(df_2.columns)
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  #print(new_column)
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+column_name ])

  #result_1 = df.join(df_2_2, how='outer')
  #print(result_1.columns)
  result = df_2_2.join(df_3, how='outer')

  return result



#process the column like (without []) 'QLD', 'climateaction', 'windenergy', 'wind', 'renewableenergy', 'olarenergy', 'cleanenergy', 'solarpower'
#make it to dummy variable and drop the dummy column if the sum in the dummy column less than 4
def seperate_tag_text_str(column_name,df,drop_number):
  #print(df.columns)
  column=df[column_name]
  my_list=[]
  #column.apply(lambda x : eval(x) if x is not np.nan else [])
  for i in column:
    tmp=[]
    #print(i)
    T_list=i.split(',')
    #print(T_list)
    for j in range(len(T_list)):
      #print(T_list[j])
      tmp.append(T_list[j])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  #return new_column
  #new_column=funct_1(column)
  vectorizer=MultiLabelBinarizer()
  #columns.fit_transform(column)
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      del df_2[column]
  df_2_2 = df.join(df_2, how='outer')
  return df_2_2


#In the json fild for the Microsoft the color column is {'dominantColorForeground': 'Grey', 'dominantColorBackground': 'Grey', 'dominantColors': ['Grey'], 'accentColor': 'B41740', 'isBwImg': False, 'isBWImg': False}
#This data extra the dominantColorForeground for the dummy variable, so on the column_name, it should be color.
def seperate_dictt_dummies(column_name,key_name,df):
  column=df[column_name]
  my_list=[]
  #column.apply(lambda x : eval(x) if x is not np.nan else [])
  for i in column:
    tmp=[]
    if i=='[]':
      tmp.append('[]')
    else:
      tmp.append(eval(i)[key_name])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  #columns.fit_transform(column)
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=key_name+'_' +vectorizer.classes_)
  tmp=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <2:
      del df_2[column]
  #print(df_2.columns)
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+key_name+column_name ])
  #print()
  result_1 = df.join(df_2, how='outer')
  result = result_1.join(df_3, how='outer')
  return result

# This code it same as the color_pickup_dominantColorForeground just extract the dominantColorBackground for the color
def color_pickup_dominantColorBackground(column_name,df):
  column=df[column_name]
  my_list=[]
  for i in column:
    tmp=[]
    tmp.append(eval(i)[dict_name])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  #columns.fit_transform(column)
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=key_name+'_' +vectorizer.classes_)
  tmp=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <2:
      del df_2[column]
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  #print(new_column)
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+key_name+column_name ])
  result_1 = df.join(df_2, how='outer')
  #print(result_1.columns)
  result = result_1.join(df_3, how='outer')
  return result

#process the the colum cell like:[{'id': 3446, 'name': 'forgiveness'}, {'id': 4399, 'name': 'victim'}, {'id': 7507, 'name': 'collage'}, {'id': 9872, 'name': 'grief'},
# {'id': 10183, 'name': 'independent film'}, {'id': 10226, 'name': 'neighbor'}, {'id': 15025, 'name': 'parenting'}]
#for instance if you want to extract the value which below  'name' and in Keywords columns such as ('forgiveness') for dummy variable.
# The drop number mean that if the sum of columns for this dummy variable less than number (in the example less than 2). This columns will be droped.
#You need set like:  seperate_dictionname_injoson('Keywords','name',df,2)
def seperate_dictionname_injoson(column_name,diction_name,df,drop_number):
  column=df[column_name]
  my_list=[]
  for i in column:
    tmp=[]
    #print(i)
    for j in range(len(eval(i))):
      tmp.append(eval(i)[j][diction_name])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      del df_2[column]
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  #print(new_column)
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+column_name ])
  result_1 = df.join(df_2, how='outer')
  result = result_1.join(df_3, how='outer')
  return result






#[{'mid': '/m/0215n', 'description': 'Cartoon', 'score': 0.9529808, 'topicality': 0.9529808}, {'mid': '/m/0jxy', 'description': 'Anime', 'score': 0.94042754, 'topicality': 0.94042754}, {'mid': '/m/02h7lkt', 'description': 'Fictional character'
#  , 'score': 0.61897975, 'topicality': 0.61897975}, {'mid': '/m/01kr8f', 'description': 'Illustration', 'score': 0.60173565, 'topicality': 0.60173565}]

def seperate_dictionname_injoson_andput_score(column_name,diction_name,diction_score_name,df,drop_number,drop_score):
  column=df[column_name]
  my_list=[]
  for i in column:
    tmp=[]
    for j in range(len(eval(i))):
      tmp.append(eval(i)[j][diction_name])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  delet_colume_name=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      delet_colume_name.append(column)
      del df_2[column]
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  #print(new_column)
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+column_name ])
  result_1 = df.join(df_2, how='outer')
  result = result_1.join(df_3, how='outer')
  column=result[column_name]
  count=0

  for i in column:
    for j in range(len(eval(i))):
      lable_name=str(column_name+'_' +eval(i)[j][diction_name])
      score=eval(i)[j][diction_score_name]
      if lable_name in delet_colume_name:
        continue
      #print(lable_name)
      if score <drop_score:
        score=0

      result.loc[count,lable_name]=score
    count=count+1

  return result


#sometimes the cells of the columns is centence like: A mother's last wishes send twins
# jeanne and Simon on a journey to Middle East in search of their tangled roots.
# Adapted from Wajdi Mouawad's acclaimed play, Incendies tells the powerful and
# moving tale of two young adults' voyage to the core of deep-rooted hatred, never-ending wars and enduring love.
#For this situation this funcation could seperat each word to a single column. But if we seperat all single to a dummy variable(single column)
#It must be creat a lot of column, so we need drop the columns if the sum in the column less than a number(you can set it).
#And there are some word such as and, we, I , show many time in the all sentence and we also need to delet it. This function could
#delet some common stop word in English.
# If I want to choice the column called text information and drop the word if it show less 20 times in the all sentence
# we do like: seperate_stencewith_simpleNLP('text',df,'20')
def seperate_stencewith_simpleNLP(column_name,df,drop_number):
  #print(df.columns)
  column=df[column_name]
  my_list=[]
  #column.apply(lambda x : eval(x) if x is not np.nan else [])
  for i in column:
    tmp=[]
    #print(i)
    T_list=i.split(' ')
    #print(T_list)
    for j in range(len(T_list)):
      #print(T_list[j])
      tmp.append(T_list[j])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  #return new_column
  #new_column=funct_1(column)
  vectorizer=MultiLabelBinarizer()
  #columns.fit_transform(column)
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      del df_2[column]
  df_2_2 = df.join(df_2, how='outer')
  nlp = spacy.load("en")
  Spacyfull_stopwords = set(nlp.Defaults.stop_words)
  full_stopwords = set(nlp.Defaults.stop_words)
  nlp = spacy.load("en")
  Spacyfull_stopwords = set(nlp.Defaults.stop_words)
  full_stopwords = set(nlp.Defaults.stop_words)
  data=df_2_2
  for name in data.columns:
    if column_name+'_' in name:
      #print(name)
      try:
        #b=re.search('keywords_(.+)', name).group(1)
        b=re.search(column_name+'_(.+)', name).group(1)
        c=b.lower()
        if c in full_stopwords:
          #print(name)
          del data[name]
      except:
        #print(name)
        del data[name]
  return data


def dummies_dic(column_name,df,drop_number,drop_score):
  column=df[column_name]
  my_list=[]
  for i in column:
    tmp=[]
    #print(i)
    for j in eval(i):
      tmp.append(j)
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  delet_colume_name=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      delet_colume_name.append(column)
      del df_2[column]
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  #print(new_column)
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+column_name ])
  result_1 = df.join(df_2, how='outer')
  result = result_1.join(df_3, how='outer')
  column=result[column_name]
  count=0

  for i in column:
    for j in eval(i):
      lable_name=str(column_name+'_' +j)
      #print(i)
      score=eval(i)[j]
      #print(score)
      if lable_name in delet_colume_name:
        continue

      if score <drop_score:
        score=0

      result.loc[count,lable_name]=score
    count=count+1

  return result


In [ ]:
from Data_Clean_Healper  import seperate_tag_text
from Data_Clean_Healper  import seperate_tag_text_str
from Data_Clean_Healper  import color_pickup_dominantColorForeground
from Data_Clean_Healper  import color_pickup_dominantColorBackground
from Data_Clean_Healper  import seperate_dictionname_injoson
from Data_Clean_Healper  import seperate_dictionname_injoson_andput_score
from Data_Clean_Healper  import seperate_stencewith_simpleNLP

In [ ]:
df = pd.read_csv("cleaned_sample_data2 (1).csv")

if you wan to make this column to  dummy variable and drop if the sum in the dummy column less than 3. you can do like:

In [ ]:
#seperate_tag_text(column_name,df,drop_number)
seperate_tag_text('tags',df,3)
#we can see we have some new columns such as 
#tags_auspol	tags_australia	tags_cleanenergy	tags_climateaction

,tags,categories_found_in_image,color,keywords1,overview,production_companies,tags_auspol,tags_australia,tags_cleanenergy,tags_climateaction,...,tags_renewableenergy,tags_renewables,tags_solar,tags_solarenergy,tags_solarpower,tags_tourism,tags_wind,tags_windenergy,tags_windpower,The sum of tags
0,"['climatechange', 'auspol']","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Grey', 'dominantC...","photography, black & white, contemporary, dese...",A mother's last wishes send twins Jeanne and S...,"[{'name': 'TS Productions', 'id': 313}, {'name...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
1,"['climatechange', 'auspol']",[],"{'dominantColorForeground': 'Brown', 'dominant...","photography, black & white, Arizona, contempor...",A biographical portrait of a pre-fame Jane Aus...,"[{'name': 'Ecosse Films', 'id': 1267}, {'name'...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
2,[],"[{'name': 'text_mag', 'score': 0.83984375}]","{'dominantColorForeground': 'Black', 'dominant...","photography, black & white, Arizona, grand can...","Andy heads off to Cowboy Camp, leaving his toy...","[{'name': 'Pixar Animation Studios', 'id': 3}]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,"['climateaction', 'nsw', 'coal', 'climatechange']","[{'name': 'others_', 'score': 0.02734375}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Oregon, contempora...",The Brothers Bloom are the best con men in the...,"[{'name': 'The Weinstein Company', 'id': 308},...",0,0,0,1,...,0,0,0,0,0,0,0,0,0,4.0
4,"['climatechange', 'auspol']","[{'name': 'text_mag', 'score': 0.8515625}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Arizona, contempor...",A married couple on the verge of separation ar...,"[{'name': 'First Point Entertainment', 'id': 7...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
5,"['climateaction', 'coal', 'climatechange', 'fo...","[{'name': 'outdoor_', 'score': 0.00390625, 'de...","{'dominantColorForeground': 'Blue', 'dominantC...","pattern, sky, blue, tall, up, windows, buildin...",A malfunctioning time machine at a ski resort ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",0,0,0,1,...,0,0,0,0,0,0,0,0,0,4.0
6,['climatechange'],"[{'name': 'outdoor_', 'score': 0.0078125, 'det...","{'dominantColorForeground': 'Orange', 'dominan...","sign, strange, surreal, woman, America, americ...","84 years later, a 101-year-old woman named Ros...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
7,"['climatechange', 'oceans']",[],"{'dominantColorForeground': 'Orange', 'dominan...","wine, women, beautifull, Toscana, dream, eroti...","When Dustin's girlfriend, Alexis, breaks up wi...","[{'name': 'Lionsgate', 'id': 1632}, {'name': '...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
8,"['climateaction', 'solarpower', 'solar', 'rene...","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Yellow', 'dominan...","Photoart black and white, Abstract bnw, floral...",As a math savant uncooks the books for a new c...,"[{'name': 'Zero Gravity Management', 'id': 535...",0,0,0,1,...,0,1,1,0,1,0,0,0,0,4.0
9,"['climatechange', 'heatwaves']","[{'name': 'text_', 'score': 0.52734375}]","{'dominantColorForeground': 'White', 'dominant...","postmodernism, tree, Israel, agriculture, family","A nanny, working for a family whose son has ju...","[{'name': 'Lakeshore Entertainment', 'id': 126...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0


In [ ]:
#No we need get the orgional data and try the next *function*
df = pd.read_csv("face_result1.csv")

In [ ]:
# for the column like 
df['keywords1']
#(without []) 'QLD', 'climateaction', 'windenergy', 'wind', 'renewableenergy', 'olarenergy', 'cleanenergy', 'solarpower'

0     photography, black & white, contemporary, dese...
1     photography, black & white, Arizona, contempor...
2     photography, black & white, Arizona, grand can...
3     photography, black & white, Oregon, contempora...
4     photography, black & white, Arizona, contempor...
5     pattern, sky, blue, tall, up, windows, buildin...
6     sign, strange, surreal, woman, America, americ...
7     wine, women, beautifull, Toscana, dream, eroti...
8     Photoart black and white, Abstract bnw, floral...
9      postmodernism, tree, Israel, agriculture, family
10    tree, building, photorealism, childrens, coupl...
11    peaceful, photography, shadows, blue, studio, ...
12    peaceful, photography, purple, red, shadows, s...
13         sea, blue, Hawaii, dance, hair, model, ocean
14    red, bikini, blue, swim, underwater, water, wo...
15    underwater, woman, dream, fantasy, fashion, fe...
16    peace, blue, underwater, yellow, dance, fashio...
17       photo, street, wall, Lanscape, dibond, 

In [ ]:
#we use the function to seperate single to dummy column and drop the dummy columns if the sum of this column less than 3
# seperate_tag_text_str(column_name,df,drop_number)
seperate_tag_text_str('keywords1',df,3)
#we can see we get some new column such as: keywords1_ Arizona	keywords1_ abstract	keywords1_ black & white	keywords1_ black and white

,tags,categories_found_in_image,color,keywords1,overview,production_companies,keywords1_ Arizona,keywords1_ abstract,keywords1_ black & white,keywords1_ black and white,...,keywords1_ love,keywords1_ nature,keywords1_ ocean,keywords1_ old,keywords1_ still life,keywords1_ surreal,keywords1_ vintage,keywords1_ woman,keywords1_black and white,keywords1_photography
0,"['climatechange', 'auspol']","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Grey', 'dominantC...","photography, black & white, contemporary, dese...",A mother's last wishes send twins Jeanne and S...,"[{'name': 'TS Productions', 'id': 313}, {'name...",0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,"['climatechange', 'auspol']",[],"{'dominantColorForeground': 'Brown', 'dominant...","photography, black & white, Arizona, contempor...",A biographical portrait of a pre-fame Jane Aus...,"[{'name': 'Ecosse Films', 'id': 1267}, {'name'...",1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,[],"[{'name': 'text_mag', 'score': 0.83984375}]","{'dominantColorForeground': 'Black', 'dominant...","photography, black & white, Arizona, grand can...","Andy heads off to Cowboy Camp, leaving his toy...","[{'name': 'Pixar Animation Studios', 'id': 3}]",1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,"['climateaction', 'nsw', 'coal', 'climatechange']","[{'name': 'others_', 'score': 0.02734375}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Oregon, contempora...",The Brothers Bloom are the best con men in the...,"[{'name': 'The Weinstein Company', 'id': 308},...",0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,"['climatechange', 'auspol']","[{'name': 'text_mag', 'score': 0.8515625}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Arizona, contempor...",A married couple on the verge of separation ar...,"[{'name': 'First Point Entertainment', 'id': 7...",1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
5,"['climateaction', 'coal', 'climatechange', 'fo...","[{'name': 'outdoor_', 'score': 0.00390625, 'de...","{'dominantColorForeground': 'Blue', 'dominantC...","pattern, sky, blue, tall, up, windows, buildin...",A malfunctioning time machine at a ski resort ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,['climatechange'],"[{'name': 'outdoor_', 'score': 0.0078125, 'det...","{'dominantColorForeground': 'Orange', 'dominan...","sign, strange, surreal, woman, America, americ...","84 years later, a 101-year-old woman named Ros...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
7,"['climatechange', 'oceans']",[],"{'dominantColorForeground': 'Orange', 'dominan...","wine, women, beautifull, Toscana, dream, eroti...","When Dustin's girlfriend, Alexis, breaks up wi...","[{'name': 'Lionsgate', 'id': 1632}, {'name': '...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"['climateaction', 'solarpower', 'solar', 'rene...","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Yellow', 'dominan...","Photoart black and white, Abstract bnw, floral...",As a math savant uncooks the books for a new c...,"[{'name': 'Zero Gravity Management', 'id': 535...",0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
9,"['climatechange', 'heatwaves']","[{'name': 'text_', 'score': 0.52734375}]","{'dominantColorForeground': 'White', 'dominant...","postmodernism, tree, Israel, agriculture, family","A nanny, working for a family whose son has ju...","[{'name': 'Lakeshore Entertainment', 'id': 126...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = pd.read_csv("data sample.csv")

In [ ]:
#For the column like 
df['color'][1]
#this kind of form from Microsoft vision color analyis most of time we need extract the dominantColorForeground and dominantColorBackground information

"{'dominantColorForeground': 'Brown', 'dominantColorBackground': 'Brown', 'dominantColors': ['Brown', 'Orange'], 'accentColor': 'CA8D01', 'isBwImg': False, 'isBWImg': False}"

In [ ]:
#so we use function:color_pickup_dominantColorBackground and function: color_pickup_dominantColorForeground to process this column
# color_pickup_dominantColorForeground(column_name,df)
df1=color_pickup_dominantColorForeground('color',df)
# color_pickup_dominantColorBackground(column_name,df)
color_pickup_dominantColorBackground('color',df1)
#we can find we have dominantColorBackground_XXX coulmns and dominantColorForeground_XXX columns

,tags,categories_found_in_image,color,keywords1,overview,production_companies,dominantColorForeground_Black,dominantColorForeground_Brown,dominantColorForeground_Grey,dominantColorForeground_Orange,...,dominantColorForeground_Yellow,The sum of dominantColorForeground color,dominantColorBackground_Black,dominantColorBackground_Blue,dominantColorBackground_Brown,dominantColorBackground_Grey,dominantColorBackground_Orange,dominantColorBackground_Red,dominantColorBackground_White,The sum of dominantColorBackgroundcolor
0,"['climatechange', 'auspol']","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Grey', 'dominantC...","photography, black & white, contemporary, dese...",A mother's last wishes send twins Jeanne and S...,"[{'name': 'TS Productions', 'id': 313}, {'name...",0,0,1,0,...,0,6.0,0,0,0,1,0,0,0,6.0
1,"['climatechange', 'auspol']",[],"{'dominantColorForeground': 'Brown', 'dominant...","photography, black & white, Arizona, contempor...",A biographical portrait of a pre-fame Jane Aus...,"[{'name': 'Ecosse Films', 'id': 1267}, {'name'...",0,1,0,0,...,0,6.0,0,0,1,0,0,0,0,6.0
2,[],"[{'name': 'text_mag', 'score': 0.83984375}]","{'dominantColorForeground': 'Black', 'dominant...","photography, black & white, Arizona, grand can...","Andy heads off to Cowboy Camp, leaving his toy...","[{'name': 'Pixar Animation Studios', 'id': 3}]",1,0,0,0,...,0,6.0,0,0,0,0,1,0,0,6.0
3,"['climateaction', 'nsw', 'coal', 'climatechange']","[{'name': 'others_', 'score': 0.02734375}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Oregon, contempora...",The Brothers Bloom are the best con men in the...,"[{'name': 'The Weinstein Company', 'id': 308},...",0,0,0,0,...,0,6.0,0,0,0,0,0,0,1,6.0
4,"['climatechange', 'auspol']","[{'name': 'text_mag', 'score': 0.8515625}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Arizona, contempor...",A married couple on the verge of separation ar...,"[{'name': 'First Point Entertainment', 'id': 7...",0,0,0,0,...,0,6.0,0,0,0,0,0,0,0,6.0
5,"['climateaction', 'coal', 'climatechange', 'fo...","[{'name': 'outdoor_', 'score': 0.00390625, 'de...","{'dominantColorForeground': 'Blue', 'dominantC...","pattern, sky, blue, tall, up, windows, buildin...",A malfunctioning time machine at a ski resort ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",0,0,0,0,...,0,6.0,0,1,0,0,0,0,0,6.0
6,['climatechange'],"[{'name': 'outdoor_', 'score': 0.0078125, 'det...","{'dominantColorForeground': 'Orange', 'dominan...","sign, strange, surreal, woman, America, americ...","84 years later, a 101-year-old woman named Ros...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",0,0,0,1,...,0,6.0,0,0,0,0,1,0,0,6.0
7,"['climatechange', 'oceans']",[],"{'dominantColorForeground': 'Orange', 'dominan...","wine, women, beautifull, Toscana, dream, eroti...","When Dustin's girlfriend, Alexis, breaks up wi...","[{'name': 'Lionsgate', 'id': 1632}, {'name': '...",0,0,0,1,...,0,6.0,0,0,0,0,1,0,0,6.0
8,"['climateaction', 'solarpower', 'solar', 'rene...","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Yellow', 'dominan...","Photoart black and white, Abstract bnw, floral...",As a math savant uncooks the books for a new c...,"[{'name': 'Zero Gravity Management', 'id': 535...",0,0,0,0,...,1,6.0,0,0,0,1,0,0,0,6.0
9,"['climatechange', 'heatwaves']","[{'name': 'text_', 'score': 0.52734375}]","{'dominantColorForeground': 'White', 'dominant...","postmodernism, tree, Israel, agriculture, family","A nanny, working for a family whose son has ju...","[{'name': 'Lakeshore Entertainment', 'id': 126...",0,0,0,0,...,0,6.0,1,0,0,0,0,0,0,6.0


In [ ]:
df = pd.read_csv("1 (1).csv")

In [ ]:
#if the columns like:
df['production_companies']

0     [{'name': 'TS Productions', 'id': 313}, {'name...
1     [{'name': 'Ecosse Films', 'id': 1267}, {'name'...
2        [{'name': 'Pixar Animation Studios', 'id': 3}]
3     [{'name': 'The Weinstein Company', 'id': 308},...
4     [{'name': 'First Point Entertainment', 'id': 7...
5     [{'name': 'United Artists', 'id': 60}, {'name'...
6     [{'name': 'Paramount Pictures', 'id': 4}, {'na...
7     [{'name': 'Lionsgate', 'id': 1632}, {'name': '...
8     [{'name': 'Zero Gravity Management', 'id': 535...
9     [{'name': 'Lakeshore Entertainment', 'id': 126...
10    [{'name': 'Paramount Pictures', 'id': 4}, {'na...
11    [{'name': 'Dimension Films', 'id': 7405}, {'na...
12    [{'name': 'Paramount Pictures', 'id': 4}, {'na...
13    [{'name': 'International Production Company', ...
14    [{'name': 'Sidetrack Films', 'id': 7426}, {'na...
15    [{'name': 'Imagine Entertainment', 'id': 23}, ...
16    [{'name': 'Roadside Attractions', 'id': 911}, ...
17    [{'name': 'Regency Enterprises', 'id': 508

In [ ]:
seperate_dictionname_injoson('production_companies','name',df,0)

In [ ]:
# For this situation we just want to extra the company name in name: to dummy columns such as Tokyo Broadcasting System, Echo Films we can use
# seperate_dictionname_injoson(column_name,diction_name,df,drop_number)
seperate_dictionname_injoson('production_companies','name',df,0)
#we can see the company name such as :production_companies_2 Entertain	production_companies_Absolute Entertainment 

,tags,categories_found_in_image,color,keywords1,overview,production_companies,production_companies_2 Entertain,production_companies_Absolute Entertainment,production_companies_Alliance Films,production_companies_Amblin Partners,...,production_companies_Vertigo Entertainment,production_companies_Village Roadshow Pictures,production_companies_Warner Bros.,production_companies_Warner Bros. Animation,production_companies_We're Not Brothers Productions,production_companies_Why Not Productions,production_companies_XYZ Films,production_companies_Yash Raj Films,production_companies_Zero Gravity Management,The sum of production_companies
0,"['climatechange', 'auspol']","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Grey', 'dominantC...","photography, black & white, contemporary, dese...",A mother's last wishes send twins Jeanne and S...,"[{'name': 'TS Productions', 'id': 313}, {'name...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
1,"['climatechange', 'auspol']",[],"{'dominantColorForeground': 'Brown', 'dominant...","photography, black & white, Arizona, contempor...",A biographical portrait of a pre-fame Jane Aus...,"[{'name': 'Ecosse Films', 'id': 1267}, {'name'...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
2,[],"[{'name': 'text_mag', 'score': 0.83984375}]","{'dominantColorForeground': 'Black', 'dominant...","photography, black & white, Arizona, grand can...","Andy heads off to Cowboy Camp, leaving his toy...","[{'name': 'Pixar Animation Studios', 'id': 3}]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3,"['climateaction', 'nsw', 'coal', 'climatechange']","[{'name': 'others_', 'score': 0.02734375}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Oregon, contempora...",The Brothers Bloom are the best con men in the...,"[{'name': 'The Weinstein Company', 'id': 308},...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
4,"['climatechange', 'auspol']","[{'name': 'text_mag', 'score': 0.8515625}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Arizona, contempor...",A married couple on the verge of separation ar...,"[{'name': 'First Point Entertainment', 'id': 7...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
5,"['climateaction', 'coal', 'climatechange', 'fo...","[{'name': 'outdoor_', 'score': 0.00390625, 'de...","{'dominantColorForeground': 'Blue', 'dominantC...","pattern, sky, blue, tall, up, windows, buildin...",A malfunctioning time machine at a ski resort ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
6,['climatechange'],"[{'name': 'outdoor_', 'score': 0.0078125, 'det...","{'dominantColorForeground': 'Orange', 'dominan...","sign, strange, surreal, woman, America, americ...","84 years later, a 101-year-old woman named Ros...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
7,"['climatechange', 'oceans']",[],"{'dominantColorForeground': 'Orange', 'dominan...","wine, women, beautifull, Toscana, dream, eroti...","When Dustin's girlfriend, Alexis, breaks up wi...","[{'name': 'Lionsgate', 'id': 1632}, {'name': '...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0
8,"['climateaction', 'solarpower', 'solar', 'rene...","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Yellow', 'dominan...","Photoart black and white, Abstract bnw, floral...",As a math savant uncooks the books for a new c...,"[{'name': 'Zero Gravity Management', 'id': 535...",0,0,0,0,...,0,0,1,0,0,0,0,0,1,4.0
9,"['climatechange', 'heatwaves']","[{'name': 'text_', 'score': 0.52734375}]","{'dominantColorForeground': 'White', 'dominant...","postmodernism, tree, Israel, agriculture, family","A nanny, working for a family whose son has ju...","[{'name': 'Lakeshore Entertainment', 'id': 126...",0,0,0,0,...,1,0,0,0,0,0,0,0,0,5.0


In [ ]:
df = pd.read_csv("face2 (1).csv")

In [ ]:
#This data from one cell in column categories_found_in_image. 
#If we want to extra the 'outdoor_' for the dummy column name and 0.00390625 for the value for this dummy column we can use function 
df['categories_found_in_image'][5]

"[{'name': 'outdoor_', 'score': 0.00390625, 'detail': {'landmarks': []}}, {'name': 'text_mag', 'score': 0.7109375}]"

In [ ]:
df=seperate_dictionname_injoson_andput_score('categories_found_in_image','name','score',df,5,0.5)

In [ ]:
df.columns

Index(['__typename', 'comments_disabled', 'dimensions', 'display_url',
       'edge_media_preview_like', 'edge_media_to_caption',
       'edge_media_to_comment', 'id', 'is_video', 'media_preview', 'owner',
       'shortcode', 'tags', 'taken_at_timestamp', 'thumbnail_resources',
       'thumbnail_src', 'urls', 'username', 'video_view_count', 'Unnamed: 0',
       'categories_found_in_image', 'color', 'tag', 'text'],
      dtype='object')

In [ ]:
#In this case we choice the 'categories_found_in_image' column and extra the value of name to make dummy column and extra the value of score for the dummy column value
#But if the score less than 0.5 the value is 0 in dummy column and if there are less 1(you can set this number by your selfe) times show in the all data set this dummy column will be drop
# seperate_dictionname_injoson_andput_score(column_name,diction_name,diction_score_name,df,drop_number,drop_score)
seperate_dictionname_injoson_andput_score('categories_found_in_image','name','score',df,1,0.5)
# We can see it make many dummy coulmn such as categories_found_in_image_text_sign and categories_found_in_image_text_mag and they have their own socre.

,tags,categories_found_in_image,color,keywords1,overview,production_companies,categories_found_in_image_abstract_,categories_found_in_image_abstract_texture,categories_found_in_image_food_,categories_found_in_image_indoor_marketstore,categories_found_in_image_others_,categories_found_in_image_outdoor_,categories_found_in_image_people_swimming,categories_found_in_image_sky_sun,categories_found_in_image_text_,categories_found_in_image_text_mag,categories_found_in_image_text_menu,categories_found_in_image_text_sign,The sum of categories_found_in_image
0,"['climatechange', 'auspol']","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Grey', 'dominantC...","photography, black & white, contemporary, dese...",A mother's last wishes send twins Jeanne and S...,"[{'name': 'TS Productions', 'id': 313}, {'name...",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.000000,0,0.519531,2.0
1,"['climatechange', 'auspol']",[],"{'dominantColorForeground': 'Brown', 'dominant...","photography, black & white, Arizona, contempor...",A biographical portrait of a pre-fame Jane Aus...,"[{'name': 'Ecosse Films', 'id': 1267}, {'name'...",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.000000,0,0.000000,0.0
2,[],"[{'name': 'text_mag', 'score': 0.83984375}]","{'dominantColorForeground': 'Black', 'dominant...","photography, black & white, Arizona, grand can...","Andy heads off to Cowboy Camp, leaving his toy...","[{'name': 'Pixar Animation Studios', 'id': 3}]",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.839844,0,0.000000,1.0
3,"['climateaction', 'nsw', 'coal', 'climatechange']","[{'name': 'others_', 'score': 0.02734375}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Oregon, contempora...",The Brothers Bloom are the best con men in the...,"[{'name': 'The Weinstein Company', 'id': 308},...",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.000000,0,0.000000,1.0
4,"['climatechange', 'auspol']","[{'name': 'text_mag', 'score': 0.8515625}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Arizona, contempor...",A married couple on the verge of separation ar...,"[{'name': 'First Point Entertainment', 'id': 7...",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.851562,0,0.000000,1.0
5,"['climateaction', 'coal', 'climatechange', 'fo...","[{'name': 'outdoor_', 'score': 0.00390625, 'de...","{'dominantColorForeground': 'Blue', 'dominantC...","pattern, sky, blue, tall, up, windows, buildin...",A malfunctioning time machine at a ski resort ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.710938,0,0.000000,2.0
6,['climatechange'],"[{'name': 'outdoor_', 'score': 0.0078125, 'det...","{'dominantColorForeground': 'Orange', 'dominan...","sign, strange, surreal, woman, America, americ...","84 years later, a 101-year-old woman named Ros...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",0,0,0.000000,0.000000,0,0,0.0000,0.773438,0.000000,0.000000,0,0.000000,2.0
7,"['climatechange', 'oceans']",[],"{'dominantColorForeground': 'Orange', 'dominan...","wine, women, beautifull, Toscana, dream, eroti...","When Dustin's girlfriend, Alexis, breaks up wi...","[{'name': 'Lionsgate', 'id': 1632}, {'name': '...",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.000000,0,0.000000,0.0
8,"['climateaction', 'solarpower', 'solar', 'rene...","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Yellow', 'dominan...","Photoart black and white, Abstract bnw, floral...",As a math savant uncooks the books for a new c...,"[{'name': 'Zero Gravity Management', 'id': 535...",0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.000000,0.000000,0,0.000000,4.0
9,"['climatechange', 'heatwaves']","[{'name': 'text_', 'score': 0.52734375}]","{'dominantColorForeground': 'White', 'dominant...","postmodernism, tree, Israel, agriculture, family","A nanny, working for a family whose son has ju...","[{'name': 'Lak

In [ ]:
df = pd.read_csv("data sample.csv")

In [ ]:
# sometimes some columns include some stences. A simple ideal is that make this 
df['overview']
#we want to extra a single word for dummy column and delet the stop word such as and, our, we.

0     A mother's last wishes send twins Jeanne and S...
1     A biographical portrait of a pre-fame Jane Aus...
2     Andy heads off to Cowboy Camp, leaving his toy...
3     The Brothers Bloom are the best con men in the...
4     A married couple on the verge of separation ar...
5     A malfunctioning time machine at a ski resort ...
6     84 years later, a 101-year-old woman named Ros...
7     When Dustin's girlfriend, Alexis, breaks up wi...
8     As a math savant uncooks the books for a new c...
9     A nanny, working for a family whose son has ju...
10    A divorced writer from the Midwest returns to ...
11    After the events at Lake Victoria, the prehist...
12    Predominantly set during World War II, Steve R...
13    After a drug-op gone bad, Joey Gazelle is put ...
14    This documentary follows the lives and careers...
15    When soldier Robin happens upon the dying Robe...
16    In a hospital on the outskirts of 1920s Los An...
17    Ally Darling (Anna Faris) is realizing she

In [ ]:
# This function could help us to extra the single word and we can see whitch kind of word could effect the regression model. 
# seperate_stencewith_simpleNLP(column_name,df,drop_number)
seperate_stencewith_simpleNLP('overview',df,5)
#we can see after drop the common connect word and stop word. And choice the frequency large than 5 times words we can get many dummy column.

,tags,categories_found_in_image,color,keywords1,overview,production_companies,overview_?€?????,overview_family,overview_find,overview_group,overview_life,overview_man,overview_mysterious,overview_set,overview_story,overview_young
0,"['climatechange', 'auspol']","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Grey', 'dominantC...","photography, black & white, contemporary, dese...",A mother's last wishes send twins Jeanne and S...,"[{'name': 'TS Productions', 'id': 313}, {'name...",0,0,0,0,0,0,0,0,0,1
1,"['climatechange', 'auspol']",[],"{'dominantColorForeground': 'Brown', 'dominant...","photography, black & white, Arizona, contempor...",A biographical portrait of a pre-fame Jane Aus...,"[{'name': 'Ecosse Films', 'id': 1267}, {'name'...",0,0,0,0,0,0,0,0,0,1
2,[],"[{'name': 'text_mag', 'score': 0.83984375}]","{'dominantColorForeground': 'Black', 'dominant...","photography, black & white, Arizona, grand can...","Andy heads off to Cowboy Camp, leaving his toy...","[{'name': 'Pixar Animation Studios', 'id': 3}]",0,0,0,0,0,0,0,0,0,0
3,"['climateaction', 'nsw', 'coal', 'climatechange']","[{'name': 'others_', 'score': 0.02734375}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Oregon, contempora...",The Brothers Bloom are the best con men in the...,"[{'name': 'The Weinstein Company', 'id': 308},...",1,0,0,0,1,0,0,0,0,0
4,"['climatechange', 'auspol']","[{'name': 'text_mag', 'score': 0.8515625}]","{'dominantColorForeground': 'White', 'dominant...","photography, black & white, Arizona, contempor...",A married couple on the verge of separation ar...,"[{'name': 'First Point Entertainment', 'id': 7...",0,0,0,0,0,0,0,0,0,0
5,"['climateaction', 'coal', 'climatechange', 'fo...","[{'name': 'outdoor_', 'score': 0.00390625, 'de...","{'dominantColorForeground': 'Blue', 'dominantC...","pattern, sky, blue, tall, up, windows, buildin...",A malfunctioning time machine at a ski resort ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",0,0,0,0,0,1,0,0,0,0
6,['climatechange'],"[{'name': 'outdoor_', 'score': 0.0078125, 'det...","{'dominantColorForeground': 'Orange', 'dominan...","sign, strange, surreal, woman, America, americ...","84 years later, a 101-year-old woman named Ros...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",0,0,0,0,1,0,0,1,1,1
7,"['climatechange', 'oceans']",[],"{'dominantColorForeground': 'Orange', 'dominan...","wine, women, beautifull, Toscana, dream, eroti...","When Dustin's girlfriend, Alexis, breaks up wi...","[{'name': 'Lionsgate', 'id': 1632}, {'name': '...",0,0,0,0,0,0,0,0,0,0
8,"['climateaction', 'solarpower', 'solar', 'rene...","[{'name': 'others_', 'score': 0.00390625}, {'n...","{'dominantColorForeground': 'Yellow', 'dominan...","Photoart black and white, Abstract bnw, floral...",As a math savant uncooks the books for a new c...,"[{'name': 'Zero Gravity Management', 'id': 535...",0,0,0,0,0,0,0,0,0,0
9,"['climatechange', 'heatwaves']","[{'name': 'text_', 'score': 0.52734375}]","{'dominantColorForeground': 'White', 'dominant...","postmodernism, tree, Israel, agriculture, family","A nanny, working for a family whose son has ju...","[{'name': 'Lakeshore Entertainment', 'id': 126...",0,1,0,0,0,0,0,0,0,0


In [ ]:
# To process the every single column in the sample data to a dummy column and download it. You can do like:
df = pd.read_csv("data sample.csv")
data1=seperate_tag_text('tags',df,3)
data2=seperate_tag_text_str('keywords1',data1,3)
data3=color_pickup_dominantColorForeground('color',data2)
data4=color_pickup_dominantColorBackground('color',data3)
data5=seperate_dictionname_injoson('production_companies','name',data4,0)
data6=seperate_dictionname_injoson_andput_score('categories_found_in_image','name','score',data5,1,0.5)
data7=seperate_stencewith_simpleNLP('overview',data6,5)

In [ ]:
frame = pd.DataFrame(data7)#To change the data mane you can download any step of data(from data1 to data7)
frame.to_csv('cleaned_sample_data.csv')
from google.colab import files
files.download("cleaned_sample_data.csv")

In [ ]:
df = pd.read_csv("face2 (1).csv")

In [ ]:
def dummees_blur(column_name,df,dropnumber):
  column=df[column_name]
  my_list=[]
  #column.apply(lambda x : eval(x) if x is not np.nan else [])
  for i in column:
    tmp=[]
    #print(i)
    if i=='[]':
      tmp.append('[]')
    else:  
      tmp.append(eval(i)['blurLevel'])
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  #columns.fit_transform(column)
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns='blurLeve'+'_' +vectorizer.classes_)
  tmp=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <dropnumber:
      del df_2[column]
  #print(df_2.columns)
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of blurLeve '+column_name ])
  #print()
  result_1 = df.join(df_2, how='outer')
  result = result_1.join(df_3, how='outer')
  return result

In [ ]:
df=color_pickup_dominantColorForeground('color',df)

In [ ]:
df=color_pickup_dominantColorBackground('color',df)

In [ ]:
df=seperate_dictionname_injoson_andput_score('categories_found_in_image','name','score',df,5,0.5)

In [ ]:
df=seperate_dictionname_injoson_andput_score('tag','name','confidence',df,5,0.5)

In [ ]:
#special_process
def dummies_dic_hair(column_name,df,drop_number,drop_score):
  column=df[column_name]
  my_list=[]
  for i in column:
    tmp=[]
    try:
      a=eval(i)['hairColor'][0]['color']
    except:
      a='None'
    tmp.append(a)
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  delet_colume_name=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      delet_colume_name.append(column)
      del df_2[column]
  column=df[column_name]
  for i in column:
    tmp.append(float(len(eval(i))))
  new_column=tmp
  #print(new_column)
  df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+column_name ])
  result_1 = df.join(df_2, how='outer')
  result = result_1.join(df_3, how='outer')
  column=result[column_name]
  count=0

  for i in column:
      tmp=[]
      #print(eval(i)['hairColor'])     
      try:
        lable_name=str(column_name+'_' +eval(i)['hairColor'][0]['color'])
        a=eval(i)['hairColor'][0]['confidence']
      except:
        lable_name='None'
        a=0
      score=a
      #print(score)
      if lable_name in delet_colume_name:
        continue

      if score <drop_score:
        score=0
      
      result.loc[count,lable_name]=score
      count=count+1

  return result


In [ ]:
from google.colab import files
files.upload()

Saving amnestyus_face_raw.csv to amnestyus_face_raw (1).csv


{'amnestyus_face_raw.csv': b',picture_name,smile,gender,age,facialHair,glasses,emotion,blur,exposure,noise,makeup,accessories,occlusion,hair\n0,[],[],[],[],[],[],[],[],[],[],[],[],[],[]\n1,14,0.997,female,40.0,"{\'moustache\': 0.0, \'beard\': 0.0, \'sideburns\': 0.0}",NoGlasses,"{\'anger\': 0.0, \'contempt\': 0.0, \'disgust\': 0.0, \'fear\': 0.0, \'happiness\': 0.997, \'neutral\': 0.003, \'sadness\': 0.0, \'surprise\': 0.0}","{\'blurLevel\': \'low\', \'value\': 0.21}","{\'exposureLevel\': \'goodExposure\', \'value\': 0.64}","{\'noiseLevel\': \'low\', \'value\': 0.25}","{\'eyeMakeup\': True, \'lipMakeup\': True}",[],"{\'foreheadOccluded\': False, \'eyeOccluded\': False, \'mouthOccluded\': False}","{\'bald\': 0.05, \'invisible\': False, \'hairColor\': [{\'color\': \'brown\', \'confidence\': 0.98}, {\'color\': \'black\', \'confidence\': 0.98}, {\'color\': \'other\', \'confidence\': 0.17}, {\'color\': \'gray\', \'confidence\': 0.17}, {\'color\': \'red\', \'confidence\': 0.11}, {\'color\': 

In [ ]:
data=pd.read_csv("amnestyus_face_raw (4).csv")

In [ ]:
df = pd.read_csv("amnestyus_face_raw (4).csv")
df=dummies_dic('emotion',df,0,0.5)
df=dummees_blur('blur',df,0)
df=seperate_dictt_dummies('exposure','exposureLevel',df)
#df=seperate_dictt_dummies('exposure','exposureLevel',df)
df=dummies_dic('occlusion',df,0,False)
df=dummies_dic('makeup',df,0,False)
df=dummies_dic_hair('hair',df,0,0.5)

In [ ]:
frame = pd.DataFrame(df)#To change the data mane you can download any step of data(from data1 to data7)
frame.to_csv('face_cleaned_data2.csv')
from google.colab import files
files.download("face_cleaned_data2.csv")

MessageError: ignored